In [1]:
import sys
import os
import numpy as np
import cv2
import openslide
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator
import xml.etree.ElementTree as ET
from xml.dom import minidom
import geojson
import argparse
import matplotlib.pyplot as plt
import fastai
from fastai.vision.all import *
import PIL
matplotlib.use('Agg')
import pandas as pd
import datetime
from skimage import draw, measure, morphology, filters
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, shape
from shapely.ops import cascaded_union, unary_union
import json
import shapely
import warnings
from scipy import ndimage
warnings.filterwarnings("ignore")

/home/jliu6/anaconda3/envs/paimg1/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/jliu6/anaconda3/envs/paimg1/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def get_y(row): return row['label'].split(" ")

In [3]:
################################################################
#######               DIR                                #######
################################################################
proj_dir = "../../"
indir1 = proj_dir + "data/OPX/"
outdir = proj_dir + "output/cancer_output/"

In [4]:
flist=sorted(glob.glob(indir1 + '*.tif'))
save_location = outdir  # args.save_location
mag_extract = [20] # do not change this, model trained at 250x250 at 20x
save_image_size = 250  # do not change this, model trained at 250x250 at 20x
run_image_size = 250 # do not change this, model trained at 250x250 at 20x
pixel_overlap = 200  # Specify the level of pixel overlap in your saved images
limit_bounds = True  # this is weird, dont change it
tiff_lvl = 2 # low res pyramid level to grab
model_path2 =  'dlv3_2ep_2e4_update-07182023_RT.pkl'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
if not os.path.exists(os.path.join(save_location)):
    os.mkdir(os.path.join(save_location))

In [6]:
# first load pytorch model
learn = load_learner(model_path2,cpu=False) 

In [8]:
for _file in flist[0:1]:
    print(_file)
    oslide = openslide.OpenSlide(_file)
    save_name = str(Path(os.path.basename(_file)).with_suffix('')) #Get sample name as save name

    if not os.path.exists(os.path.join(save_location,save_name+'_tissue.json')):
        # This is physical microns per pixel
        acq_mag = 10.0 / float(oslide.properties[openslide.PROPERTY_NAME_MPP_X])

        # this is nearest multiple of 20 for base layer
        base_mag = int(20 * round(float(acq_mag) / 20))

        # this is how much we need to resample our physical patches for uniformity across studies
        physSize = round(save_image_size * acq_mag / base_mag)

        # # grab tiles accounting for the physical size we need to pull for standardized tile size across studies
        # tiles = DeepZoomGenerator(oslide, tile_size=physSize - round(self.pixel_overlap * acq_mag / base_mag),
        #                           overlap=round(self.pixel_overlap * acq_mag / base_mag / 2),
        #                           limit_bounds=self.limit_bounds)

        # # calculate the effective magnification at each level of tiles, determined from base magnification
        # tile_lvls = tuple(base_mag / (tiles._l_z_downsamples[i] * tiles._l0_l_downsamples[tiles._slide_from_dz_level[i]]) for i in range(0, tiles.level_count))

        # # intermeadiate level for probability map
        # lvl_img = oslide.read_region((0, 0), self.tiff_lvl, oslide.level_dimensions[self.tiff_lvl])
        # lvl_resize = oslide.level_downsamples[self.tiff_lvl]
        # lvl_img.save(os.path.join(self.save_location,save_name+'_low-res.png'))
                

../../data/OPX/OPX_001.tif


In [9]:
properties = oslide.properties #microns per pixel in the X dimension of level 0
df = pd.DataFrame(dict(properties.items()), index = [0])
df

,openslide.comment,openslide.level-count,openslide.level[0].downsample,openslide.level[0].height,openslide.level[0].tile-height,openslide.level[0].tile-width,openslide.level[0].width,openslide.level[1].downsample,openslide.level[1].height,openslide.level[1].tile-height,...,ventana.Mode,ventana.ScanMode,ventana.ScanRes,ventana.ScanWhitePoint,ventana.ScannerModel,ventana.ShowLabel,ventana.UnitNumber,ventana.UserName,ventana.Z-layers,ventana.Z-spacing
0,level=0 mag=40 quality=95,8,1,19456,1024,1024,24576,2,9728,1024,...,brightfield,1,0.25,235,VENTANA DP 200,1,2000440,Operator,1,1


In [12]:
print(oslide.level_dimensions)

((24576, 19456), (12288, 9728), (6144, 4864), (3072, 2432), (1536, 1216), (768, 608), (384, 304), (192, 152))


In [10]:
properties

<_PropertyMap {'openslide.comment': 'level=0 mag=40 quality=95', 'openslide.level-count': '8', 'openslide.level[0].downsample': '1', 'openslide.level[0].height': '19456', 'openslide.level[0].tile-height': '1024', 'openslide.level[0].tile-width': '1024', 'openslide.level[0].width': '24576', 'openslide.level[1].downsample': '2', 'openslide.level[1].height': '9728', 'openslide.level[1].tile-height': '1024', 'openslide.level[1].tile-width': '1024', 'openslide.level[1].width': '12288', 'openslide.level[2].downsample': '4', 'openslide.level[2].height': '4864', 'openslide.level[2].tile-height': '1024', 'openslide.level[2].tile-width': '1024', 'openslide.level[2].width': '6144', 'openslide.level[3].downsample': '8', 'openslide.level[3].height': '2432', 'openslide.level[3].tile-height': '1024', 'openslide.level[3].tile-width': '1024', 'openslide.level[3].width': '3072', 'openslide.level[4].downsample': '16', 'openslide.level[4].height': '1216', 'openslide.level[4].tile-height': '1024', 'opensli

In [13]:
acq_mag = 10.0 / float(0.25) ##0.25 mircro per pixel
acq_mag

40.0

In [14]:
base_mag = int(20 * round(acq_mag / 20)) # round it into the nearest base 10 number (In case the acq_mag is not int, maybe can use this round(101.111, -1))
base_mag

40

In [ ]:
physSize = round(save_image_size * acq_mag / base_mag)
physSize

In [ ]:
save_image_size*acq_mag /base_mag

In [ ]:
#At a 40X magnification, it takes a grid of 4 x 4 pixels to represent 1 square micrometer. -> 0.25 MPP (micro per pixel), or 4PPM (pixel per micro)

In [ ]:
# grab tiles accounting for the physical size we need to pull for standardized tile size across studies
tiles = DeepZoomGenerator(oslide, tile_size=physSize - round(pixel_overlap * acq_mag / base_mag),
                          overlap=round(pixel_overlap * acq_mag / base_mag / 2),
                          limit_bounds=limit_bounds)

In [ ]:
physSize - round(pixel_overlap * acq_mag / base_mag)
round(pixel_overlap * acq_mag / base_mag / 2)

In [ ]:
# calculate the effective magnification at each level of tiles, determined from base magnification
tile_lvls = tuple(base_mag / (tiles._l_z_downsamples[i] * tiles._l0_l_downsamples[tiles._slide_from_dz_level[i]]) for i in range(0, tiles.level_count))
tile_lvls

In [ ]:
print("The number of levels in the tiles object are : ", tiles.level_count)
print("The dim of in each level are : ", tiles.level_dimensions) #pixel dim
print("total number of tiles = ", tiles.tile_count)

In [ ]:
# grab tiles accounting for the physical size we need to pull for standardized tile size across studies
tiles2 = DeepZoomGenerator(oslide, tile_size=1024,overlap=0,limit_bounds=limit_bounds)
print("The number of levels in the tiles object are : ", tiles2.level_count)
print("The dim of in each level are : ", tiles2.level_dimensions) #pixel dim
print("total number of tiles = ", tiles2.tile_count)

In [ ]:
df


In [ ]:
tiles = DeepZoomGenerator(oslide, tile_size=50, overlap=100, limit_bounds=False)
print("The number of levels in the tiles object are : ", tiles.level_count)
print("The pixel dim of in each level are : ", tiles.level_dimensions) #pixel dim
print("total number of tiles = ", tiles.tile_count)

selected_tiles = tiles
#tiles at a specific level
total_tile_counts = 0
for l in range(0,selected_tiles.level_count):
    #level_tiles[k] are the tile counts of level k.
    x_counts = selected_tiles.level_tiles[l][0]
    y_counts = selected_tiles.level_tiles[l][1]
    cur_total = x_counts*y_counts
    print("Tiles counts at X and Y dim at level ", l, " is: ", selected_tiles.level_tiles[l]) #pix dim a level/250 = count (count number of tile images at 250 x 250)
    print("This means there are ", x_counts*y_counts, "tiles in total") #counts in x times counts in y
    total_tile_counts += x_counts*y_counts
print("final total is: ",total_tile_counts)

In [ ]:
rows, cols = tiles.level_tiles[-1]
print(rows, cols)

In [ ]:
print("Width",24576/50)
print("Height",19456/50)

In [ ]:
#
row = 210
col = 210
single_tile = tiles.get_tile(tiles.level_count - 1,(row,col))
single_tile_RGB = single_tile.convert('RGB')
singel_tile_np = np.array(single_tile_RGB)
print(singel_tile_np.shape)
single_tile_RGB